## Web Scraping

### Rate My Professor: http://www.ratemyprofessors.com/search.jsp

In [1]:
import requests
import requests_cache

from bs4 import BeautifulSoup
import json
import pandas as pd
from http import cookiejar
from time import sleep
import random
from lxml import html
import re
import sqlite3 as sql

requests_cache.install_cache('professor_cache')

In [2]:
import sqlite3 as sql
db = sql.connect('data.db')

In [3]:
# get professors' names through 2018 Winter courses we scrapped

professor_raw = pd.read_sql('SELECT * FROM course',db)
professor_raw = professor_raw['cname']
professor_name_raw = []
for name in professor_raw:
    professor_name_raw.append(name.split('-')[-1].replace(',', ''))
    
professor_name_all = set(professor_name_raw)
professor_name_all.remove('')
professor_name_all.remove('The Staff')
professor_name_all = [i for i in professor_name_all]
len(professor_name_all)

3085

In [12]:
begin = int(input("Enter begin #:"))
end = int(input("Enter end #:"))

Enter begin #:1
Enter end #:100


In [13]:
# part 1
url = 'http://www.ratemyprofessors.com/search.jsp'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
           'Host': 'api.bizographics.com'}
professor_rate = pd.DataFrame(columns = ['Professor', 'Department', 'date', 'Class', 'For Credit', 'Attendance', 'Textbook Used',
               'Would Take Again', 'Grade Received', 'RatingType', 'Overall Quality',
               'Level of Difficulty', 'comment tags', 'comment'])

# get a list of names of professors who cannot be found on the website
professor_none =[]

In [17]:
# part 2
professor_name = professor_name_all[(begin-1):end]
for name in professor_name:
    ##query is the name of the professor
    params = {'queryoption': 'HEADER','queryBy':'teacherName',"schoolName":'University of California Davis',
             'schoolID':'1073','query':name}
    response = requests.get(url, params=params)##, headers=headers)
    
    bs = BeautifulSoup(response.text,"lxml")
    
    ##find the rating-website of the professor
    location = bs.find("li",{'class':'listing PROFESSOR'})
    if location == None:
        professor_none.append(name)
        print("Can't find the Professor: " +str(name))
    else:
        website = location.find("a")["href"]
        newurl = "http://www.ratemyprofessors.com" +website
        print(newurl)
    
    ##get the page source of the new url
        responsepro = requests.get(newurl)
    
        bspro = BeautifulSoup(responsepro.text,"lxml")
    
    # get professor's name
        prf_name = {"Professor": name}
        print(prf_name)
    
    # get department name
        if bspro.find("div",{"class":'result-title'}) != None:
            (dept,uni) = bspro.find("div",{"class":'result-title'}).get_text().strip().split("at" or "in")[0:2]
            uni = uni.split(",")[0].strip()
            dept = dept.strip()
            dept = dept.strip('Professor in ')
            dept = {"Department": dept}
    
    #get overall quality
        if bspro.find("div",{"class":'breakdown-header'}) != None:
            overall_quality = bspro.find("div",{"class":'breakdown-header'}).get_text().split("Quality")[1].strip()
    
    # get takeagain
        if bspro.find("div",{"class":'breakdown-section takeAgain no-rating'}) != None:
            takeagain = bspro.find("div",{"class":'breakdown-section takeAgain no-rating'}).get_text().split("Again")[1].strip()
    
    # get level of difficulty
        if bspro.find("div",{"class":'breakdown-section difficulty'}) != None:
            level_of_difficulty = bspro.find("div",{"class":'breakdown-section difficulty'}).get_text().split("Difficulty")[1].strip()
    
    
    #### need to scrape by using regex
        if bspro.find('div',{'class':'rating-filter togglable'}) != None:
            place = bspro.find('div',{'class':'rating-filter togglable'})
            table = place.find('table', {'class':'tftable'})
            rows = table.findAll('tr')
    
        for row in rows:  ##row1 = students1
            if row.find('td',{'class':'rating'}) != None:
                cols = row.findAll('td')
                cols = [ele.text.strip() for ele in cols]
                overall = {}
                
                #add professor
                overall.update(prf_name)
        
                # add apartment name
                overall.update(dept)
            
                # add date inside using regrex
                date = re.search("([0-9]{2}/){2}[0-9]{4}", cols[0]).group()
                overall.update({'date': date})

            # add first part (must be after above date part)
                cols0 = cols[0].split('\n')
                cols0 = [x for x in cols0 if x != '']
                cols0.remove(date)
                cols0.insert(1, 'RatingType')
                cols0 = dict(zip(cols0[1::2], cols0[::2]))
                overall.update(cols0)

            # add second part
                cols1 = cols[1].split('\n')
                cols1 = [a.split(':') for a in cols1]
                cols1[0].insert(0, 'Class')  # prepare a dictionary for later dataframe
                overall.update(dict(cols1))

            # add comment tags inside

                class_comments = row.find('td',{'class':'comments'})
                comment_tags = class_comments.find('div',{'class':'tagbox'})
                comment_tags = comment_tags.text.strip().split('\n')
                overall.update({'comment tags': comment_tags})

            # add comment contents

                comment_content = class_comments.find('p',{'class':'commentsParagraph'})
                comment_content = comment_content.text.strip('\r\n').strip()
                overall.update({'comment':comment_content})

            # get the dataframe
                df_overall = pd.DataFrame.from_dict(overall, orient='index')
                df_overall = df_overall.reset_index().transpose()
                df_overall = df_overall.rename(columns = df_overall.iloc[0]).drop(df_overall.index[0]).reset_index(drop = True)
                professor_rate = pd.concat([professor_rate,df_overall], axis=0)

Can't find the Professor: Lein Pamela
Can't find the Professor: Quinn James
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=162400
{'Professor': 'Ranganath Nicole'}
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=788029
{'Professor': 'Pierre'}
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=26112
{'Professor': 'Gravner Janko'}
Can't find the Professor: BAKERJIAN
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=317899
{'Professor': 'Burnett Katharine'}
Can't find the Professor: Spyridakis S
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=639267
{'Professor': 'Warden Craig'}
Can't find the Professor: NICHOLS K
Can't find the Professor: Henwood Kathryn
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=783210
{'Professor': 'Bender Melissa'}
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=364212
{'Professor': 'Rosenberg Moshe'}
http://www.ratemyprofessors.com/ShowRatings.jsp?tid=2254434
{'Professor': 'St John Rachel'}
Can't find the Professor: Diflorio Elaine
http:/

In [40]:
print(professor_none)
with open("professor_didnt_find.txt", "w") as output:
    output.write(str(professor_none))

['Warner Tobias', 'Wang Zhongren', 'Haudenschild Domini', 'Holsberry Kimberly', 'Brown Patrick', 'Delmer Deborah', 'HERCZEG LIU', 'Philipps Anthony', 'Rosenquist Grace', 'Gorin Fredric', 'Circella Giovanni', 'Baldwin Roger', 'Stuart Kathleen', 'Tal Gil', 'Kellogg Cynthia', 'Sudarshana Mysore', 'Temple J.', 'Bahre C', 'Pessah Isaac', 'Grandia Elizabeth', 'Favia Dominic', 'Guy Brenda', 'GILMORE C', 'Wallace Matthew', 'Sutcliffe Julie', 'Frankel Edwin', 'Van Eenennaam Aliso', 'Mitchell Rebecca', 'Boone John', 'Hotaling Jason', 'Carter Cameron', 'Stern Joshua', 'Sanders Seth', 'Houston Pamela', 'Kornbluth Kurt', 'Hyde Timothy', 'Shauman Kimberlee', 'Forrest Alexander']


In [41]:
professor_rate.to_csv("ratemyprofessor("+str(begin)+"-"+str(end)+").csv")